# Cortex Game Python Demo-Round 2_Prob

Import SASpy and connect to your SAS studio session


(Instruction of SASpy install and configuaration:
https://support.sas.com/ondemand/saspy.html )


In [1]:
from saspy import SASsession
sas_session = SASsession()
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 2082



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /Users/sunxinyi/opt/anaconda3/lib/python3.8/site-packages/saspy/sascfg_personal.py
WORK Path             = /saswork/SAS_workDC330001DAAE_odaws01-usw2.oda.sas.com/SAS_work45B00001DAAE_odaws01-usw2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 3.7.2
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 121518


Load raw datasets from SAS Studio (Change the folder path to where your "Cortex Data Sets" folder located.)

In [2]:
%%SAS sas_session

libname cortex '/home/u58717790/my_shared_file_links/u39842936/Cortex Data Sets';

# Step1-2 Merge and DataPartition
link for reference: 

Pandas library: https://pandas.pydata.org/docs/user_guide/index.html


sklearn.model_selection: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [3]:
import pandas as pd

#comment: Transform cloud sas dataset to python datafrmae(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

In [4]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge.head()
#Deal with Missing Value 
data_merge = data_merge.dropna()  #comment: you could use another method to deal with NA
data_merge.head()

#Step2 Data Partition 
#this is just a sample, you could use another library or bulit in funciton
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.4) # you can change the percentage
train.head()

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
632463,2632464.0,GRANT,MIGUEL,0.0,58.0,195100.0,University / College,Suburban,10.0,5.0,...,4.0,8.0,85.0,20.0,25.0,0.0,0.0,0.0,1.0,30.0
362806,2362807.0,LEAR,ROSALINDA,1.0,23.0,6100.0,High School,Suburban,2.0,1.0,...,1.0,2.0,10000.0,10000.0,10000.0,0.0,0.0,0.0,0.0,0.0
736423,2736424.0,GREER,ELVIRA,1.0,36.0,33200.0,University / College,City,8.0,3.0,...,3.0,7.0,550.0,20.0,500.0,0.0,0.0,0.0,1.0,30.0
151824,2151825.0,EGAN,RONALD,0.0,23.0,46000.0,University / College,City,2.0,0.0,...,1.0,1.0,20.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0
177096,2177097.0,DAMBROSIO,DOROTHY,1.0,19.0,8500.0,University / College,Downtown,2.0,0.0,...,1.0,1.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0


# Step3 Linear Regression Model


link for reference:

sk-learn library: https://scikit-learn.org/stable/index.html

In [5]:
from sklearn import linear_model

#comment: it's numpy array ==> categorical variables
X_train = train[['Age', 'Salary','Seniority', 'GaveLastYear','Contact']] 
Y_train = train['GaveThisYear']
X_valid = validation[['Age', 'Salary','Seniority', 'GaveLastYear','Contact']] 
Y_valid = validation['GaveThisYear']

regr = linear_model.LinearRegression()
regr.fit(X_train,Y_train)
regr_predict=regr.predict(X_valid)

In [6]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

0.3115840986168856
0.15557273443306519
0.3944270964741966


# Step4 Decision Tree Model（Py）

In [7]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age','Salary','Seniority','GaveLastYear','Contact']] 
Y_train = train['GaveThisYear']
X_valid = validation[['Age','Salary','Seniority','GaveLastYear','Contact']] 
Y_valid = validation['GaveThisYear']

DT_model = DecisionTreeRegressor(max_depth=5).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[0.15691794 0.16074115 0.18059014 ... 0.23067916 0.18059014 0.16074115]


In [8]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

0.308987636839947
0.15438671347281632
0.39292074706334396


### Other models may also be helpful for this game:
Link for reference:
1. Logistic regression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression
2. Neural Net: https://scikit-learn.org/stable/modules/classes.html?highlight=neural%20net#module-sklearn.neural_network       

# Step6 Scoring the data

Pick the best model from previous steps and use it to predict for next year donation.

In [9]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='SCORE_RD2_NOCONTACT',
libref='cortex'
)

In [10]:
#Predict prob of give given contact

scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")
scoring_data_contact = scoring_data_contact.dropna() 
scoring_data_contact.head()

X = scoring_data_contact[['Age','Salary','Seniority','GaveLastYear','Contact']] 
regr_predict_contact=regr.predict(X)
scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'ProbContact'}, axis=1) 
scoring_data_contact.head()

,ID,ProbContact
0,2000001.0,0.366904
9,2000010.0,0.351865
14,2000015.0,0.399713
15,2000016.0,0.513633
16,2000017.0,0.465812


In [11]:
#Predict prob of give given not contact

scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")
scoring_data_nocontact = scoring_data_nocontact.dropna() 
scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age','Salary','Seniority','GaveLastYear','Contact']] 
regr_predict_nocontact=regr.predict(X)
scoring_data_nocontact['Prediction'] = regr_predict_nocontact
scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'ProbNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,ProbNoContact
0,2000001.0,0.154752
9,2000010.0,0.139714
14,2000015.0,0.187562
15,2000016.0,0.301482
16,2000017.0,0.253660


In [12]:
result = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result.sort_values(by=['ID'], inplace=True)
result.head()

,ID,ProbContact,ProbNoContact
0,2000001.0,0.366904,0.154752
1,2000010.0,0.351865,0.139714
2,2000015.0,0.399713,0.187562
3,2000016.0,0.513633,0.301482
4,2000017.0,0.465812,0.253660


# Step7 Exporting CSV (Py)

In [15]:
result.to_csv('Round2_Output_prob.csv', index=False)